In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

%matplotlib notebook

In [2]:

%matplotlib notebook

In [3]:
effl_df = 0

with open("NI03/NI03.eof", "r") as f:
    f_str = f.read()
    effl_start_semi = f_str.rfind("[EFFLUENT]")
    effl_start = f_str.find(" ", effl_start_semi)
    effl_end = f_str.rfind("[EFFLUENT END]")
    effl_str = f_str[effl_start+1 : effl_end]
    print(effl_str)
    d = re.findall(r"([^\s]+)\s([0-9\.]+)", effl_str)

    effl_df = pd.DataFrame(d, columns=["Component", "Value"])
effl_df = effl_df.set_index("Component")
effl_df["Value"].astype('float64')
effl_df

WH2 1.104631 WCH4 17.755289 WC2H2 0.652727 WC2H4 33.193439 WC2H6 3.184381 WC3H4 0.863027 WC3H6 13.553437 WC3H8 0.322236 WNBUTA 0.083413 WIBUTA 0.022578 WIB 1.769784 WB1 0.897014 WB2 0.547584 WBUTAD 4.830544 WBENZ 8.474632 WTOLUO 2.535606 WXILO 0.417858 WEBENZ 0.069887 WSTYR 1.185272 WC9ARO 0.032798 WNC5 0.388231 WIC5 0.216596 WNC6 0.086474 WNC7 0.004056 WNC8 0.000287 WNC9 0.000063 WNC10 0.000006 WUND1 0.000002 WCPTAN 0.04013 WMCPTA 0.03559 WCESAN 0.048971 WC7NAF 0.0039 WC8NAF 0.000095 WC9NAF 0.000016 WB2M2 0.005101 WP1 0.051151 WFUEL1 1.481004 WB1M2 0.074387 WMMCSE 0.007607 WC10IS 0 WPTD 0.35902 WISOPR 0.255078 WAOLC6 0.009735 WMCESE 0.051418 WC6ISO 0.042053 WESIJ 0.013552 WPIJMK 0.13872 WAOLC7 0.001065 WMCP 0.033395 WC7ISO 0.00381 WEIJMK 0.039277 WCPTD 1.080204 WAOLC8 0.000106 WCC5H8 0.262051 WC8ISO 0.000204 WC9ISO 0.000049 WCESEN 0.088795 WDIAL 0.009584 WC4H4 0.11946 WMSTYR 0.310979 WDMSTY 0.038824 WFUEL2 0.022528 WFUEL3 1.167826 WMCPTD 0.19131 WC7H10 0.052136 WINDEN 0.284477 WFUEL4 

,Value
Component,
WH2,1.104631
WCH4,17.755289
WC2H2,0.652727
WC2H4,33.193439
WC2H6,3.184381
...,...
WC3MEA,0.518917
WPRPD,0.34411
VC3MEA,0.348722


In [4]:
effluent = {}

effluent_list = ["wt", "vol", "RC4","RC4-pygas", "Pygas", "H/C ratio", "MW", "misc"]
for effluent_sublist in effluent_list:
    effluent[effluent_sublist] = {}

for component in effl_df.index:
    if component[0] == "W":
        effluent["wt"][component[1:]] = effl_df.loc[component, 'Value']
    elif component[0] == "V":
        effluent["vol"][component[1:]] = effl_df.loc[component, 'Value']
    elif component[0] == "R":
        effluent["RC4-pygas"][component[1:]] = effl_df.loc[component, 'Value']
    elif component[0: 2] == "HC":
        effluent["H/C ratio"][component[2:]] = effl_df.loc[component, 'Value']
    elif component[0: 2] == "MW":
        effluent["MW"][component[2:]] = effl_df.loc[component, 'Value']
    else:
        effluent["misc"][component] = effl_df.loc[component, 'Value']
        
for effluent_sub in effluent:
    effluent[effluent_sub] = pd.Series(effluent[effluent_sub]).astype("float64")

/tmp/ipykernel_211763/3635209071.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  effluent[effluent_sub] = pd.Series(effluent[effluent_sub]).astype("float64")


In [5]:
 raw_c4_comps = [
     "C4H4",
     "BUTAD",
     "B1",
     "B2C",
     "B2T",
     "IB",
     "NBUTA",
     "IBUTA",
 ]
effluent["RC4"] = effluent["RC4-pygas"].loc[raw_c4_comps]
effluent["Pygas"] = effluent["RC4-pygas"].loc[~effluent["RC4-pygas"].index.isin(raw_c4_comps)]

In [6]:
print(effluent["RC4"])
print(effluent["RC4"].sum())

C4H4      1.444433
BUTAD    58.407776
B1       10.846108
B2C       2.780833
B2T       3.840199
IB       21.399067
NBUTA     1.008574
IBUTA     0.273002
dtype: float64
99.999992


In [7]:
print(effluent["Pygas"])
print(effluent["Pygas"].sum())

C5PAR    21.611309
P1        1.827685
P2        1.430043
B1M2      2.657955
B1M3      0.953622
B2M2      0.182273
PTD      12.828297
ISOPR     9.114282
CPTAN     1.433902
CC5H8     9.363451
CPTD     38.597187
dtype: float64
100.00000599999998


In [8]:
effluent["misc"]

MPRAT       1.310021
PERAT       0.408317
C3ERAT      0.404980
STEAM      39.220291
AMWWET     22.533987
CONVKEY     0.983763
PLACE       6.000000
dtype: float64

In [9]:
pd.concat([effluent["wt"], effluent["vol"]], axis=1, sort=False, keys=['wt', 'vol'])

,wt,vol
H2,1.104631,14.751690
CH4,17.755289,29.797802
C2H2,0.652727,0.674952
C2H4,33.193439,31.856897
C2H6,3.184381,2.851252
...,...,...
C1724,0.989161,0.124659
C25+,1.068734,0.068310
C5C9,16.915237,5.600940
C3MEA,0.518917,0.348722


In [14]:
general_spyro = " TEMPWA 1107.211792 TEGENERALMPGA 1235.53064 VELOLI 137.85379 VELOMA 62.028976 TEMPCO 4.789137 AVTMT 945.782532 TOTDUT 4616664 HEATFA 71822.898438 VOLINS 1127.422729 SURINS 64.278442 SUROUT 77.243156 COILWE 3929.578857 INLETP 2.173671 TXADIA 842.00293 CIT 589 MASSFL 5999.999512 DSRATIO 0.65 CONOP 1 COKVOL 0"

In [17]:
data = general_spyro.split()
parameters = data[::2]
values = data[1::2]

In [18]:
pd.DataFrame(values, parameters)

,0
TEMPWA,1107.211792
TEGENERALMPGA,1235.53064
VELOLI,137.85379
VELOMA,62.028976
TEMPCO,4.789137
AVTMT,945.782532
TOTDUT,4616664
HEATFA,71822.898438
VOLINS,1127.422729
SURINS,64.278442
